# Mid-term Project

In [1531]:
# Importing necessary libraries
import os
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json
import datetime
import mysql.connector
from sqlalchemy import create_engine, text
import sqlalchemy
import pymongo
import certifi


In [1533]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.30
Running PyMongo Version: 4.10.1


In [1535]:
mysql_args = {
    "uid" : "root",
    "pwd" : "unicornTree98*!",
    "hostname" : "localhost",
    "dbname" : "sakila"
}

src_dbname = "sakila"
dst_dbname = "sakila_dw"

mongodb_args = {
    "user_name" : "paige7",
    "password" : "OxzQQK8aGjaUHcgT",
    "cluster_name" : "Cluster0",
    "cluster_subnet" : "vl73d",
    "cluster_location" : "atlas", # "local"
    "db_name" : "sakila"}

In [1537]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [1539]:
conn_str = f"mysql+pymysql://{mysql_args['uid']}:{mysql_args['pwd']}@{mysql_args['hostname']}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

In [1541]:
sakila_staff = "SELECT * FROM sakila.staff;"
df_staff = get_sql_dataframe(sakila_staff, **mysql_args)
df_staff.head(2)

,staff_id,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-02-15 03:57:16
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,None,2006-02-15 03:57:16


In [1543]:
sakila_film = "SELECT * FROM sakila.film;"
df_film = get_sql_dataframe(sakila_film, **mysql_args)
df_film.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


In [1545]:
sakila_rentals = "SELECT * FROM sakila.rental;"
df_rentals = get_sql_dataframe(sakila_rentals, **mysql_args)
df_rentals.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


In [1546]:
sakila_payment = "SELECT * FROM sakila.payment;"
df_payment = get_sql_dataframe(sakila_payment, **mysql_args)
df_payment.head(2)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30


In [1547]:
sakila_store = "SELECT * FROM sakila.store;"
df_store = get_sql_dataframe(sakila_store, **mysql_args)
df_store.head(2)

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [1549]:
sakila_customer = "SELECT * FROM sakila.customer;"
df_customer = get_sql_dataframe(sakila_customer, **mysql_args)
df_customer.head(2)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [1553]:
client = get_mongo_client(**mongodb_args)

# Current Working Directory
data_dir = os.path.join(os.getcwd(), 'Downloads')

json_files = {"inventory" : 'sakila-inventory.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)  

In [1554]:
client = get_mongo_client(**mongodb_args)

query = {} 
collection = "inventory"

df_inventory = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_inventory.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [1555]:
data_dir = os.path.join(os.getcwd(), 'downloads')
data_file = os.path.join(data_dir, 'sakila-payment.csv')
df_payment = pd.read_csv(data_file, sep=";", header=0, index_col=0)
df_payment.head()

,customer_id,staff_id,rental_id,amount,payment_date,last_update
payment_id,,,,,,
1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [1556]:
# Transformations

In [1557]:
drop_cols = ['address_id','picture','password', 'last_update', 'store_id']
df_staff.drop(drop_cols, axis=1, inplace=True)
df_staff.rename(columns={"staff_id":"staff_key"}, inplace=True)

df_staff.head(2)

,staff_key,first_name,last_name,email,active,username
0,1,Mike,Hillyer,Mike.Hillyer@sakilastaff.com,1,Mike
1,2,Jon,Stephens,Jon.Stephens@sakilastaff.com,1,Jon


In [1558]:
drop_cols = ['language_id','original_language_id', 'last_update']
df_film.drop(drop_cols, axis=1, inplace=True)
df_film.rename(columns={"film_id":"film_key"}, inplace=True)
df_inventory.rename(columns={"inventory_id": "inventory_key"}, inplace=True)

df_film.head(2)

,film_key,title,description,release_year,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,3,4.99,48,12.99,G,"Trailers,Deleted Scenes"


In [1559]:
drop_cols = []
df_store.drop(drop_cols, axis=1, inplace=True)
df_store.rename(columns={"store_id":"store_key"}, inplace=True)
df_store.head(2)

df_inventory.rename(columns={"store_id": "store_key"}, inplace=True)


In [1560]:
drop_cols = ['store_id','address_id','last_update']
df_customer.drop(drop_cols, axis=1, inplace=True)
df_customer.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_customer.head(2)

df_customer.rename(columns={"customer_id": "customer_key"}, inplace=True)

In [1561]:
drop_cols = ['last_update']
df_inventory.drop(drop_cols, axis=1, inplace=True)
df_inventory.rename(columns={"inventory_id":"inventory_key"}, inplace=True)

df_inventory.head(20)

,inventory_key,film_id,store_key
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,2
5,6,1,2
6,7,1,2
7,8,1,2
8,9,2,2
9,10,2,2


In [1562]:
drop_cols = ['last_update', 'return_date']
df_rentals.drop(drop_cols, axis=1, inplace=True)
df_rentals.rename(columns={"rental_id":"rental_key", "inventory_id":"inventory_key", "customer_id":"customer_key","staff_id":"staff_key"}, inplace=True)
df_rentals.head(2)

,rental_key,rental_date,inventory_key,customer_key,staff_key
0,1,2005-05-24 22:53:30,367,130,1
1,2,2005-05-24 22:54:33,1525,459,1


In [1563]:
drop_cols = ['last_update', 'customer_id']
df_payment.drop(drop_cols, axis=1, inplace=True)
df_payment.rename(columns={"rental_id":"rental_key", "payment_id":"payment_key"}, inplace=True)
df_payment.head(2)

,staff_id,rental_key,amount,payment_date
payment_id,,,,
1,1,76,2.99,2005-05-25 11:30:37
2,1,573,0.99,2005-05-28 10:35:23


In [1564]:
print(df_rentals.columns)
print(df_payment.columns)

Index(['rental_key', 'rental_date', 'inventory_key', 'customer_key',
       'staff_key'],
      dtype='object')
Index(['staff_id', 'rental_key', 'amount', 'payment_date'], dtype='object')


In [1565]:
df_fact_rentals_ = pd.merge(df_rentals, df_payment, on='rental_key', how='inner')
df_fact_rentals_.rename(columns={"amount":"payment_amount"}, inplace=True)

df_fact_rentals_.head(2)

,rental_key,rental_date,inventory_key,customer_key,staff_key,staff_id,payment_amount,payment_date
0,1,2005-05-24 22:53:30,367,130,1,1,2.99,2005-05-24 22:53:30
1,2,2005-05-24 22:54:33,1525,459,1,2,2.99,2005-05-24 22:54:33


In [1566]:
print(df_fact_rentals_.columns)
print(df_staff.columns)

Index(['rental_key', 'rental_date', 'inventory_key', 'customer_key',
       'staff_key', 'staff_id', 'payment_amount', 'payment_date'],
      dtype='object')
Index(['staff_key', 'first_name', 'last_name', 'email', 'active', 'username'], dtype='object')


In [1567]:
# Join staff to rentals on the staff key
df_fact_rentals = pd.merge(df_fact_rentals_, df_staff, on='staff_key', how='left')
df_fact_rentals.drop(['staff_key'], axis=1, inplace=True)
df_fact_rentals.rename(columns={"address_id":"address_key"}, inplace=True)
df_fact_rentals.head(3).sort_values(by=['rental_key'])

,rental_key,rental_date,inventory_key,customer_key,staff_id,payment_amount,payment_date,first_name,last_name,email,active,username
0,1,2005-05-24 22:53:30,367,130,1,2.99,2005-05-24 22:53:30,Mike,Hillyer,Mike.Hillyer@sakilastaff.com,1,Mike
1,2,2005-05-24 22:54:33,1525,459,2,2.99,2005-05-24 22:54:33,Mike,Hillyer,Mike.Hillyer@sakilastaff.com,1,Mike
2,3,2005-05-24 23:03:39,1711,408,2,3.99,2005-05-24 23:03:39,Mike,Hillyer,Mike.Hillyer@sakilastaff.com,1,Mike


In [1568]:
# Date Dimension

In [1569]:
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(5)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02
2,20000103,2000-01-03
3,20000104,2000-01-04
4,20000105,2000-01-05


In [ ]:
db_operation = "insert"
tables = [('dim_film', df_film, 'film_key')]

for table_name, dataframe, primary_key in tables:
    set_dataframe(uid, pwd, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [ ]:
# Count number of times each movie was rented

sql_query = '''
SELECT dim_films. AS 'Movie',
COUNT(*) AS 'Number of Rentals'
FROM dim_films 
LEFT OUTER JOIN fact_rentals 
ON dim_films.film_key = fact_rentals.film_key
GROUP BY dim_films.title;
'''
test = get_sql_dataframe(sql_query, **mysql_args)